In [ ]:
# Nate Brunacini, nbrunaci@u.rochester.edu
# Supervisor: Kelly A. Douglass
# This file runs scatterplotgenerator over all galaxies in the MaNGA dataset and writes the resulting gradient, 
# y-intercept, and other values to a text file. It also creates a histogram of slopes for each metallicity method 
# from Brown et al. 2016 (N2, O3N2, and N2O2).

In [1]:
# Import packages
from scatterplotgenerator import *
import time
from astropy.io import ascii

/home/nbrunaci/.local/lib/python3.9/site-packages/marvin/core/exceptions.py:50: UserWarning: cannot initiate Sentry error reporting: [Errno 6] No such device or address.
  warnings.warn('cannot initiate Sentry error reporting: {0}.'.format(str(ee)),
[INFO]: No release version set. Setting default to DR15
[WARNING]: path /home/nbrunaci/sas/mangawork/manga/spectro/redux/v2_4_3/drpall-v2_4_3.fits cannot be found. Setting drpall to None. (MarvinUserWarning)
[WARNING]: path /home/nbrunaci/sas/mangawork/manga/spectro/analysis/v2_4_3/2.2.1/dapall-v2_4_3-2.2.1.fits cannot be found. Setting dapall to None. (MarvinUserWarning)


In [2]:
params_list = []
total_galaxies = 0
successes = 0# Number of galaxies for which data was available
exceptions = 0# Number of galaxies for which data was not available

# Open and close FitParameters.txt every time; that might fix the disconnect/~memory issue

with fits.open('drpall-v2_4_3.fits', memmap=True) as drpall:
    for i in range(0,1):# Temporary; just to fix indentation issue quickly
        data = drpall[1].data
        c_table = Table.read('DRP-master_file_vflag_BB_smooth1p85_mapFit_N2O2_HIdr2_noWords_v5.txt',format='ascii.commented_header')
        start = time.time()
        for index in range(4987,len(data['plateifu'])):# Use this to iterate over every galaxy
#         for index in range(0,1):
            total_galaxies += 1
            try:
#             for i in range(0,1):# For testing only
                plateifu = data['plateifu'][index]
                plate,ifu = plateifu.split('-')
                fit_parameters = find_gradient(plateifu,data,c_table)
                N2p = fit_parameters['N2_params']
                O3N2p = fit_parameters['O3N2_params']
                N2O2p = fit_parameters['N2O2_params']
#                 params_list.append([index,plateifu,N2p.slope,N2p.intercept,N2p.rvalue,N2p.pvalue,N2p.stderr,N2p.intercept_stderr,
#                                       O3N2p.slope,O3N2p.intercept,O3N2p.rvalue,O3N2p.pvalue,O3N2p.stderr,O3N2p.intercept_stderr,
#                                       N2O2p.slope,N2O2p.intercept,N2O2p.rvalue,N2O2p.pvalue,N2O2p.stderr,N2O2p.intercept_stderr])
                addition= [index,plateifu,N2p.slope,N2p.intercept,N2p.rvalue,N2p.pvalue,N2p.stderr,N2p.intercept_stderr,
                                      O3N2p.slope,O3N2p.intercept,O3N2p.rvalue,O3N2p.pvalue,O3N2p.stderr,O3N2p.intercept_stderr,
                                      N2O2p.slope,N2O2p.intercept,N2O2p.rvalue,N2O2p.pvalue,N2O2p.stderr,N2O2p.intercept_stderr]
                with open('FitParameters.txt','a') as param_file:
                    param_file.write('\n')
                    for element in addition:#params_list[len(params_list)-1]:
                        param_file.write(str(element) + ' ')
#                     param_file.write('\n' + str(params_list[len(params_list)-1]))
                successes += 1
            except Exception:
                exceptions += 1
        end = time.time()
        print('Number of galaxies for which data could not be retrieved:',str(exceptions))
        print('Elapsed time:',str(end - start))
        # Checking for issues
        if not exceptions+successes == total_galaxies:
            print('Exceptions+successes does not equal total_galaxies')
    
#     # Determine and print the number of galaxies with negative metallicity gradient
#     negatives = 0
#     for i in range(0,len(gradient_list)):
#         if gradient_list[i]['slope_N2'] < 0.0:
#             negatives += 1
#             print('Index:',i)
#             print('Slope:',gradient_list[i]['slope_N2'],'\n')
#     print('Number of galaxies with negative metallicity gradient:',negatives,'out of',total_galaxies)
#     print('Fraction of galaxies with negative metallicity gradient:',negatives/total_galaxies)

[WARNING]: OverflowError converting to FloatType in column avg_alpha, possibly resulting in degraded precision. (AstropyWarning)
[WARNING]: OverflowError converting to FloatType in column pos_alpha, possibly resulting in degraded precision. (AstropyWarning)
[WARNING]: OverflowError converting to FloatType in column neg_alpha, possibly resulting in degraded precision. (AstropyWarning)
[WARNING]: invalid value encountered in arccos (RuntimeWarning)
[WARNING]: invalid value encountered in arccos (RuntimeWarning)
[WARNING]: invalid value encountered in arccos (RuntimeWarning)
[WARNING]: invalid value encountered in arccos (RuntimeWarning)
[WARNING]: invalid value encountered in arccos (RuntimeWarning)
[WARNING]: invalid value encountered in arccos (RuntimeWarning)
[WARNING]: invalid value encountered in arccos (RuntimeWarning)
[WARNING]: invalid value encountered in arccos (RuntimeWarning)
[WARNING]: invalid value encountered in arccos (RuntimeWarning)
[WARNING]: invalid value encountered 

Number of galaxies for which data could not be retrieved: 3820
Elapsed time: 1859.8811161518097


In [13]:
# Number of galaxies for which data could not be retrieved: 53
# Elapsed time: 22.54923415184021 

# Index: 0
# Slope: -0.07078364428412859 

# Index: 13
# Slope: -0.2734158900293699 

# Index: 32
# Slope: -0.05130774420463602 

# Index: 37
# Slope: -0.03218017080577616 

# Index: 42
# Slope: -0.1035295400220195 

# Number of galaxies with negative metallicity gradient: 5 out of 47
# Fraction of galaxies with negative metallicity gradient: 0.10638297872340426

In [3]:
# Count how many galaxies have negative trend line slopes for N2 and output their index, plateifu, and slope

with open('FitParameters.txt','r') as param_file:
    header = param_file.readline()
    count = 0# Number of galaxies with negative slope
    large_negatives = 0# Number of galaxies with slope less than -0.1
    large_negatives_plateifu = []
    for index in range(1,4717):
        line = param_file.readline().split(' ')
        if float(line[2]) < 0:
            count += 1
#             print('Index:',line[0])
#             print('Plateifu:',line[1])
#             print('Slope:',line[2]+'\n')
            if float(line[2]) < -0.1:
                large_negatives += 1
                large_negatives_plateifu.append(line[1])
    print('Number of galaxies with negative slope:',count)
    print('Fraction of galaxies with negative slope:',count/4717)
#     print('Number of galaxies with slope less than -0.1:',large_negatives)
#     print('Fraction of galaxies with slope less than -0.1:',large_negatives/4717)
#     print(large_negatives_plateifu)

Number of galaxies with negative slope: 760
Fraction of galaxies with negative slope: 0.16111935552257792


In [ ]:
# Results from second-order N2 method:
# Number of galaxies with negative slope: 760
# Fraction of galaxies with negative slope: 0.16111935552257792

# Results from linear N2 method:
# Number of galaxies with negative slope: 751
# Fraction of galaxies with negative slope: 0.15921136315454737

# Possibly good galaxies:
# 7992-12701
# 8445-3701 Probably not this one. The normalized radius values are negative

In [4]:
# Create histogram of slopes for each metallicity method

data = ascii.read('FitParameters.txt')
N2_slope = data.columns['N2_slope']
O3N2_slope = data.columns['O3N2_slope']
N2O2_slope = data.columns['N2O2_slope']

# print(min(N2_slope))
# print(max(N2_slope))

plt.hist(N2_slope,bins=np.linspace(-1,2,100))#,bins,alpha=0.5,weights=weights[3],label='SDSS DR7',color='gray',density=True)
plt.xlabel('Slope of Line of Best Fit')
plt.ylabel('Number of Galaxies')
plt.title('N2 Metallicity Gradient of MaNGA Galaxies')
plt.savefig('PosterMaps_Updated/SlopeHistogram_N2')
plt.close()

plt.hist(O3N2_slope,bins=np.linspace(-1,2,100))#,bins,alpha=0.5,weights=weights[3],label='SDSS DR7',color='gray',density=True)
plt.xlabel('Slope of Line of Best Fit')
plt.ylabel('Number of Galaxies')
plt.title('O3N2 Metallicity Gradient of MaNGA Galaxies')
plt.savefig('PosterMaps_Updated/SlopeHistogram_O3N2')
plt.close()

plt.hist(N2O2_slope,bins=np.linspace(-1,2,100))#,bins,alpha=0.5,weights=weights[3],label='SDSS DR7',color='gray',density=True)
plt.xlabel('Slope of Line of Best Fit')
plt.ylabel('Number of Galaxies')
plt.title('N2O2 Metallicity Gradient of MaNGA Galaxies')
plt.savefig('PosterMaps_Updated/SlopeHistogram_N2O2')
plt.close()

[WARNING]: OverflowError converting to FloatType in column N2_pvalue, possibly resulting in degraded precision. (AstropyWarning)
[WARNING]: OverflowError converting to FloatType in column O3N2_pvalue, possibly resulting in degraded precision. (AstropyWarning)
[WARNING]: OverflowError converting to FloatType in column N2O2_pvalue, possibly resulting in degraded precision. (AstropyWarning)


-298.53894409600423
730.7919167970337


In [8]:
# data[0]['N2_slope']

-0.07078364428412853